Este archivo contiene el procedimiento para generar el dataset inicial a partir de la base de datos de [Hugo Mathien](https://www.kaggle.com/datasets/hugomathien/soccer).

In [1]:
import sqlite3
import os
from time import time

import numpy as np
import pandas as pd

### Funciones para extraer datos

In [2]:
def get_match_label(match: pd.Series):
    """Devuelve el resultado (label/etiqueta) de un partido."""

    home_goals = match["home_team_goal"]
    away_goals = match["away_team_goal"]

    labels = pd.DataFrame()
    labels.loc[0, "match_api_id"] = match["match_api_id"]

    if home_goals > away_goals:
        labels.loc[0, "label"] = "Home"
    if home_goals == away_goals:
        labels.loc[0, "label"] = "Draw"
    if home_goals < away_goals:
        labels.loc[0, "label"] = "Away"

    return labels.loc[0]


In [3]:
def get_fifa_player_stats(match: pd.Series, player_stats: pd.DataFrame):
    """Devuelve el atributo 'Overall' de cada jugador para un partido"""

    match_id = match["match_api_id"]
    date = match["date"]
    players = [
        "home_player_1",
        "home_player_2",
        "home_player_3",
        "home_player_4",
        "home_player_5",
        "home_player_6",
        "home_player_7",
        "home_player_8",
        "home_player_9",
        "home_player_10",
        "home_player_11",
        "away_player_1",
        "away_player_2",
        "away_player_3",
        "away_player_4",
        "away_player_5",
        "away_player_6",
        "away_player_7",
        "away_player_8",
        "away_player_9",
        "away_player_10",
        "away_player_11",
    ]
    player_stats_new = pd.DataFrame()
    names = []

    for player in players:
        player_id = match[player]
        stats: pd.Series = player_stats[player_stats["player_api_id"] == player_id]

        current_stats: pd.Series = stats[stats["date"] < date].sort_values(
            by="date", ascending=False
        )[:1]

        if np.isnan(player_id):
            overall_rating = pd.Series(0)
        else:
            current_stats.reset_index(inplace=True, drop=True)
            overall_rating = pd.Series(current_stats.loc[0, "overall_rating"])

        name = f"{player}_overall_rating"
        names.append(name)

        player_stats_new = pd.concat([player_stats_new, overall_rating], axis=1)

    player_stats_new.columns = names
    player_stats_new["match_api_id"] = match_id

    player_stats_new.reset_index(inplace=True, drop=True)

    return player_stats_new.iloc[0]


In [4]:
def get_fifa_team_stats(match: pd.Series, team_stats: pd.DataFrame):
    """Devuelve los atributos de FIFA de los equipos para un solo partido."""

    match_id = match["match_api_id"]
    date = match["date"]

    teams = ["home_team", "away_team"]

    class_names = [
        "buildUpPlaySpeed",
        "buildUpPlaySpeedClass",
        "buildUpPlayDribblingClass",
        "buildUpPlayPassing",
        "buildUpPlayPassingClass",
        "buildUpPlayPositioningClass",
        "chanceCreationPassing",
        "chanceCreationPassingClass",
        "chanceCreationCrossing",
        "chanceCreationCrossingClass",
        "chanceCreationShooting",
        "chanceCreationShootingClass",
        "chanceCreationPositioningClass",
        "defencePressure",
        "defencePressureClass",
        "defenceAggression",
        "defenceAggressionClass",
        "defenceTeamWidth",
        "defenceTeamWidthClass",
        "defenceDefenderLineClass",
    ]

    team_stats_new = pd.DataFrame()

    for team_name in teams:
        team = team_name + "_api_id"
        one_team_stats = pd.DataFrame()
        one_team_names = []

        for col in class_names:
            team_id = match[team]
            stats: pd.Series = team_stats[team_stats["team_api_id"] == team_id]

            current_stats: pd.Series = stats[stats["date"] < date].sort_values(
                by="date", ascending=False
            )[:1]

            if current_stats.shape[0] == 0:
                current_stats: pd.Series = stats.sort_values(by="date", ascending=True)[
                    :1
                ]

            if np.isnan(team_id):
                curr_stat = pd.Series(0)
            else:
                try:
                    current_stats.reset_index(inplace=True, drop=True)
                    curr_stat = pd.Series(current_stats.loc[0, col])
                except:
                    curr_stat = pd.Series(0)

            new_name = f"{team_name}_{col}"
            one_team_names.append(new_name)

            one_team_stats = pd.concat([one_team_stats, curr_stat], axis=1)

        one_team_stats.columns = one_team_names
        one_team_stats["match_api_id"] = match_id

        team_stats_new = pd.concat([team_stats_new, one_team_stats], axis=1)

    return team_stats_new.iloc[0]


In [5]:
def get_fifa_data(
    matches: pd.DataFrame, player_stats: pd.DataFrame, team_stats: pd.DataFrame
):
    """Devuelve los atributos de FIFA para jugadores y partidos."""

    print("Recoleccion de data de FIFA")

    print("Recolectando data de equipos...")
    start = time()
    player_data = matches.apply(lambda x: get_fifa_team_stats(x, team_stats), axis=1)
    end = time()
    print(
        "Data de equipos de FIFA recolectada en {:.1f} minutos.".format(
            (end - start) / 60
        )
    )

    print("Recolectando data de jugadores...")
    start = time()
    team_data = matches.apply(lambda x: get_fifa_player_stats(x, player_stats), axis=1)
    end = time()
    print(
        "Data de jugadores de FIFA recolectada en {:.1f} minutos.".format(
            (end - start) / 60
        )
    )

    return player_data, team_data


In [6]:
def get_last_matches(matches: pd.DataFrame, date, team, x: int = 10):
    """Devuelve los últimos x partidos de un equipo anteriores a una fecha."""

    team_matches: pd.DataFrame = matches[
        (matches["home_team_api_id"] == team) | (matches["away_team_api_id"] == team)
    ]

    last_matches = (
        team_matches[team_matches["date"] < date]
        .sort_values(by="date", ascending=False)
        .iloc[0:x, :]
    )

    return last_matches


In [7]:
def get_last_matches_against_eachother(
    matches: pd.DataFrame, date, home_team, away_team, x: int = 3
):
    """Devuelve los últimos x partidos entre dos equipos anteriores a una fecha."""

    home_matches = matches[
        (matches["home_team_api_id"] == home_team)
        & (matches["away_team_api_id"] == away_team)
    ]
    away_matches = matches[
        (matches["home_team_api_id"] == away_team)
        & (matches["away_team_api_id"] == home_team)
    ]
    total_matches = pd.concat([home_matches, away_matches], axis=0)

    try:
        last_matches: pd.DataFrame = (
            total_matches[total_matches["date"] < date]
            .sort_values(by="date", ascending=False)
            .iloc[0:x, :]
        )
    except:
        last_matches: pd.DataFrame = (
            total_matches[total_matches["date"] < date]
            .sort_values(by="date", ascending=False)
            .iloc[0 : total_matches.shape[0], :]
        )

        if last_matches.shape[0] > x:
            print("Error al recuperar partidos entre dos equipos.")

    return last_matches


In [8]:
def get_goals(matches: pd.DataFrame, team) -> int:
    """Devuelve la cantidad de goles anotados por un equipo en un conjunto de partidos."""

    home_goals = int(
        matches["home_team_goal"][matches["home_team_api_id"] == team].sum()
    )
    away_goals = int(
        matches["away_team_goal"][matches["away_team_api_id"] == team].sum()
    )

    total_goals = home_goals + away_goals

    return total_goals


def get_goals_conceided(matches: pd.DataFrame, team) -> int:
    """Devuelve los goles recibidos por un equipo en un conjunto de partidos."""

    home_goals = int(
        matches["home_team_goal"][matches["away_team_api_id"] == team].sum()
    )
    away_goals = int(
        matches["away_team_goal"][matches["home_team_api_id"] == team].sum()
    )

    total_goals = home_goals + away_goals

    return total_goals


In [9]:
def get_wins(matches: pd.DataFrame, team) -> int:
    """Obtiene # de victorias de un equipo en x partidos."""

    home_wins = int(
        matches["home_team_goal"][
            (matches["home_team_api_id"] == team)
            & (matches["home_team_goal"] > matches["away_team_goal"])
        ].count()
    )
    away_wins = int(
        matches["away_team_goal"][
            (matches["away_team_api_id"] == team)
            & (matches["away_team_goal"] > matches["home_team_goal"])
        ].count()
    )

    total_wins = home_wins + away_wins

    return total_wins


In [10]:
def get_match_features(match: pd.Series, matches: pd.DataFrame, x: int = 10):
    """Devuelve los atributos finales para un único partido."""

    date = match["date"]
    home_team = match["home_team_api_id"]
    away_team = match["away_team_api_id"]

    matches_home_team = get_last_matches(matches, date, home_team, x=x)
    matches_away_team = get_last_matches(matches, date, away_team, x=x)

    last_matches_against = get_last_matches_against_eachother(
        matches, date, home_team, away_team, x=3
    )

    home_goals = get_goals(matches_home_team, home_team)
    away_goals = get_goals(matches_away_team, away_team)
    home_goals_conceided = get_goals_conceided(matches_home_team, home_team)
    away_goals_conceided = get_goals_conceided(matches_away_team, away_team)

    result = pd.DataFrame()

    result.loc[0, "match_api_id"] = match["match_api_id"]
    result.loc[0, "league_id"] = match["league_id"]

    result.loc[0, "home_team_goals"] = home_goals
    result.loc[0, "home_team_goals_conceided"] = home_goals_conceided
    result.loc[0, "away_team_goals"] = away_goals
    result.loc[0, "away_team_goals_conceided"] = away_goals_conceided

    result.loc[0, "home_team_goals_mean"] = home_goals / x
    result.loc[0, "away_team_goals_mean"] = away_goals / x

    result.loc[0, "home_team_goals_difference"] = home_goals - home_goals_conceided
    result.loc[0, "away_team_goals_difference"] = away_goals - away_goals_conceided

    result.loc[0, "home_team_games_won"] = get_wins(matches_home_team, home_team)
    result.loc[0, "away_team_games_won"] = get_wins(matches_away_team, away_team)

    result.loc[0, "home_team_games_won_against"] = get_wins(
        last_matches_against, home_team
    )
    result.loc[0, "away_team_games_won_against"] = get_wins(
        last_matches_against, away_team
    )

    return result.loc[0]


### Funciones para generar atributos

In [11]:
def generate_probs(match_odds: pd.DataFrame):
    """Convierte los factores de apuesta en probabilidades para fácil lectura."""

    match_id = match_odds.loc[:, "match_api_id"]
    bookkeeper = match_odds.loc[:, "bookkeeper"]
    win_odd = match_odds.loc[:, "Win"]
    draw_odd = match_odds.loc[:, "Draw"]
    loss_odd = match_odds.loc[:, "Defeat"]

    win_prob = 1 / win_odd
    draw_prob = 1 / draw_odd
    loss_prob = 1 / loss_odd

    total_prob = win_prob + draw_prob + loss_prob

    probs = pd.DataFrame()

    probs.loc[:, "match_api_id"] = match_id
    probs.loc[:, "bookkeeper"] = bookkeeper
    probs.loc[:, "Win"] = win_prob / total_prob
    probs.loc[:, "Draw"] = draw_prob / total_prob
    probs.loc[:, "Defeat"] = loss_prob / total_prob

    return probs


def get_bets_data(matches: pd.DataFrame, bookkeepers: pd.DataFrame, horizontal=True):
    """Agrega la data de apuestas para los partidos."""

    bk_data = pd.DataFrame()

    for bookkeeper in bookkeepers:

        temp_data = matches.loc[:, (matches.columns.str.contains(bookkeeper))]
        temp_data.loc[:, "bookkeeper"] = str(bookkeeper)
        temp_data.loc[:, "match_api_id"] = matches.loc[:, "match_api_id"]

        cols = temp_data.columns.values
        cols[:3] = ["Win", "Draw", "Defeat"]
        temp_data.columns = cols
        temp_data.loc[:, "Win"] = pd.to_numeric(temp_data["Win"])
        temp_data.loc[:, "Draw"] = pd.to_numeric(temp_data["Draw"])
        temp_data.loc[:, "Defeat"] = pd.to_numeric(temp_data["Defeat"])

        if horizontal == True:

            temp_data = generate_probs(temp_data)
            temp_data.drop("match_api_id", axis=1, inplace=True)
            temp_data.drop("bookkeeper", axis=1, inplace=True)

            win_name = bookkeeper + "_" + "Win"
            draw_name = bookkeeper + "_" + "Draw"
            defeat_name = bookkeeper + "_" + "Defeat"
            temp_data.columns.values[:3] = [win_name, draw_name, defeat_name]

            bk_data = pd.concat([bk_data, temp_data], axis=1)
        else:
            bk_data = bk_data.append(temp_data, ignore_index=True)

    if horizontal == True:
        temp_data.loc[:, "match_api_id"] = matches.loc[:, "match_api_id"]

    return bk_data


def generate_odds(matches, bets, horizontal=True):
    """Genera los datos de las apuetas."""

    print("Generando data de apuestas...")
    start = time()

    bets_data = get_bets_data(matches, bets, horizontal=horizontal)

    bets_data.loc[:, "match_api_id"] = matches.loc[:, "match_api_id"]
    end = time()
    print("Data de apuestas generada en {:.1f} minutos.".format((end - start) / 60))

    return bets_data


In [12]:
def generate_matches(matches: pd.DataFrame, x: int = 10):
    """Genera los atributos de las estadísticas de los partidos."""

    print("Generado atributos de partidos")
    start = time()

    match_stats = matches.apply(
        lambda match: get_match_features(match, matches, x=x), axis=1
    )
    dummies = pd.get_dummies(match_stats["league_id"]).rename(
        columns=lambda x: "is_league_" + str(int(x))
    )
    match_stats = pd.concat([match_stats, dummies], axis=1)
    match_stats.drop(["league_id"], inplace=True, axis=1)

    end = time()
    print("Data de partidos generada en {:.1f} minutos.".format((end - start) / 60))

    return match_stats


In [13]:
def generate_labels(matches: pd.DataFrame):
    """Genera las etiquetas (labels) para todos los partidos."""

    print("Generando etiquetas...")
    start = time()

    labels = matches.apply(get_match_label, axis=1)

    end = time()
    print("Etiquetas generadas en {:.1f}".format((end - start) / 60))

    return labels


In [14]:
def generate_dataset(
    matches: pd.DataFrame,
    players: pd.DataFrame,
    teams: pd.DataFrame,
    bets: pd.DataFrame,
    labels: pd.DataFrame,
):
    """Crea el dataset final"""

    features = pd.merge(matches, players, on="match_api_id", how="left")
    features = pd.merge(features, teams, on="match_api_id", how="left")
    features = pd.merge(features, bets, on="match_api_id", how="left")
    features = pd.merge(features, labels, on="match_api_id", how="left")

    features.dropna(inplace=True)

    return features


### Procedimiento

In [17]:
# conexion con la BD y obtencion de los datos
database = "../data/database.sqlite"
conn = sqlite3.connect(database)

player_data = pd.read_sql("SELECT * FROM Player;", conn)
player_stats_data = pd.read_sql("SELECT * FROM Player_Attributes;", conn)
team_stats_data = pd.read_sql("SELECT * FROM Team_Attributes", conn)
team_data = pd.read_sql("SELECT * FROM Team;", conn)
match_data = pd.read_sql("SELECT * FROM Match;", conn)


In [18]:
preserve_rows = [
    "country_id",
    "league_id",
    "season",
    "stage",
    "date",
    "match_api_id",
    "home_team_api_id",
    "away_team_api_id",
    "home_team_goal",
    "away_team_goal",
    "home_player_1",
    "home_player_2",
    "home_player_3",
    "home_player_4",
    "home_player_5",
    "home_player_6",
    "home_player_7",
    "home_player_8",
    "home_player_9",
    "home_player_10",
    "home_player_11",
    "away_player_1",
    "away_player_2",
    "away_player_3",
    "away_player_4",
    "away_player_5",
    "away_player_6",
    "away_player_7",
    "away_player_8",
    "away_player_9",
    "away_player_10",
    "away_player_11",
]
match_data.dropna(subset=preserve_rows, inplace=True)


In [19]:
# atributos de partidos, equipos y jugadores
match_stats = generate_matches(match_data)
team_stats, player_stats = get_fifa_data(match_data, player_stats_data, team_stats_data)
team_stats = team_stats.loc[:, ~team_stats.columns.duplicated()]


Generado atributos de partidos
Data de partidos generada en 4.3 minutos.
Recoleccion de data de FIFA
Recolectando data de equipos...
Data de equipos de FIFA recolectada en 15.3 minutos.
Recolectando data de jugadores...
Data de jugadores de FIFA recolectada en 10.6 minutos.


In [20]:
# atributos de apuestas
bookkeper_preserve = ["B365", "BW", "IW", "LB", "WH", "VC"]
bets_stats = generate_odds(match_data, bookkeper_preserve)


Generando data de apuestas...
Data de apuestas generada en 0.0 minutos.


/tmp/ipykernel_1692/1788728731.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data.loc[:, "bookkeeper"] = str(bookkeeper)
/tmp/ipykernel_1692/1788728731.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_data.loc[:, "match_api_id"] = matches.loc[:, "match_api_id"]
/tmp/ipykernel_1692/1788728731.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [21]:
# etiquetas y generacion de dataset
labels = generate_labels(match_data)
dataset = generate_dataset(match_stats, player_stats, team_stats, bets_stats, labels)


Generando etiquetas...
Etiquetas generadas en 0.3


In [23]:
dataset.to_csv("../data/dataset_initial.csv")
